# Local AI-powered search system

The code builds a local AI-powered search system. It lets you ask questions about a collection of documents, and the AI will respond using only those documents to answer you.

You get a reliable, document-grounded answer to your question — all done locally using your own files and models.

The user_query field is where you enter the particular question you have

In [1]:
# Install dependencies

# Installs the `unstructured` package, which provides tools for parsing and extracting data from unstructured documents such as PDFs, text files, and images.
!pip install unstructured

# Installs `faiss-cpu`, a CPU-only version of FAISS (Facebook AI Similarity Search). FAISS is used for efficient similarity search and clustering of high-dimensional vectors, commonly used for tasks like information retrieval and recommendation systems.
!pip install faiss-cpu

# Installs `langchain`, an open-source framework for building applications that integrate with language models (such as GPT). It provides a unified interface for working with AI models, document loaders, and other tools in your workflows.
!pip install langchain

# Installs the `langchain_community` package, which contains additional community-contributed components for LangChain, including custom document loaders, retrievers, and other enhancements that are not part of the core LangChain library.
!pip install langchain_community

# Installs `langchain_ollama`, a package that enables integration with Ollama’s local models for language model-based tasks. It allows you to run models like `phi-4-mini` locally without the need for an API call.
!pip install langchain_ollama

# Installs `python-magic-bin`, a precompiled binary version of `python-magic` for Windows. `python-magic` is a Python interface to the `libmagic` file type detection library, which helps automatically detect file types based on file content. This is useful when working with mixed document types (e.g., PDFs, text files) in tasks like document loading.
!pip install python-magic-bin

!pip install pymupdf


### Part 1: Prepare the documents and setup the vectorstore

Load files from a folder by reading files from a local folder on your computer.

Break them into smaller pieces - since documents can be long, chop them into smaller, overlapping chunks for easier processing.

Convert text into numbers (embeddings) - use a local model (via Ollama) to turn each chunk into a special set of numbers that represent the meaning of the text. This is called an embedding.

Store these embeddings in a searchable index - save everything into a database (called a vector store, using FAISS) so it can later find similar chunks when you ask a question.

Save it for later use - the index is saved to your computer so you don’t have to rebuild it each time.

In [2]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyMuPDFLoader

# Define a custom loader for mixed file types (txt, csv, pdf)
class CustomDocumentLoader:
    def __init__(self, path, glob="**/*.{txt,csv,pdf}"):
        self.path = path
        self.glob = glob
    
    def load(self):
        # Manually load files from the directory using the glob pattern
        documents = []
        
        # Recursively traverse the directory and check file extensions
        for root, _, files in os.walk(self.path):
            for file in files:
                file_path = os.path.join(root, file)
                if file.endswith('.txt'):
                    # Load text files
                    with open(file_path, 'r') as f:
                        documents.append(f.read())  # Add text content
                elif file.endswith('.csv'):
                    # Use CSVLoader for CSV files
                    csv_loader = CSVLoader(file_path=file_path)
                    documents.extend(csv_loader.load())  # Add CSV content
                elif file.endswith('.pdf'):
                    # Use PyMuPDFLoader for PDF files
                    pdf_loader = PyMuPDFLoader(file_path=file_path)
                    documents.extend(pdf_loader.load())  # Add PDF content
        
        return documents

# Use the CustomDocumentLoader to load .txt, .csv, and .pdf files
loader = CustomDocumentLoader(r"C:\Python\Agent-School\docs\docs2")
docs = loader.load()

# Now `docs` contains documents from .txt, .csv, and .pdf files
print(f"📄 Loaded {len(docs)} documents.")


📄 Loaded 51 documents.


In [3]:
# Setup Vectorstore

from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Load local text/PDF/Markdown/etc. files
loader = DirectoryLoader(r"C:\Python\Agent-School\docs\docs2", glob="**/*.txt")
docs = loader.load()

# 2. Split into manageable chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 3. Create embeddings using Ollama's local model
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# 4. Create FAISS index
vectorstore = FAISS.from_documents(chunks, embedding_model)

# 5. Save locally
vectorstore.save_local("faiss_index_docs2")

print("✅ Vector store created and saved.")


✅ Vector store created and saved.


### Part 2: Ask a question and get an answer (rag_phi4_query)
Load the saved vector store
It loads the document database created earlier.

Find the most relevant chunks
When you ask a question (like "Describe the Apollo programme"), it searches the index and pulls out the top 5 chunks most related to your question.

Create a grounded prompt
It puts those chunks into a prompt that tells the AI:
“Only use this information to answer — don’t make things up.”

Ask a local language model (phi4-mini)
It sends the prompt to a small AI model running locally (no internet needed), and the model generates a thoughtful answer based only on the context it was given.

In [4]:
# Perform RAG query

from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
import ollama

# 1. Load FAISS vector store
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.load_local(
    "faiss_index_docs2",
    embedding_model,
    allow_dangerous_deserialization=True
)


# 2. Retrieve top-k relevant chunks
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

user_query = "Describe the Apollo programme "
docs = retriever.invoke(user_query)

# 3. Construct grounded prompt
context = "\n\n".join([doc.page_content for doc in docs])
prompt = f"""
You are a helpful assistant.

Use only the information in the context below to answer the question.
If the answer is not in the context, say:
"I don't have enough information to answer that."

Context:
{context}

Question: {user_query}
Answer:
"""

# 4. Ask phi-4-mini via Ollama
response = ollama.chat(
    model="phi4-mini",
    messages=[{"role": "user", "content": prompt}]
)

print("\n🧠 Answer:")
print(response["message"]["content"])



🧠 Answer:
The Apollo program is one of United States' most iconic achievements in human history that aimed for astronauts to travel into outer-space. Announced by President John F. Kennedy on May 25th, 1961 during his State of Union Address at Rice University he said “We choose to go to the moon.” The first manned mission was Apollo 7 launched July 11-14 and achieved a major milestone when astronaut Wally Schirra made an orbital flight around Earth.

The program culminated in September 26th, 1969 with Neil Armstrong becoming humanity’s first space explorer by landing on lunar surface. In addition to being the third manned mission of Apollo Program NASA also launched two unmanned spacecrafts namely Lunar Orbiter and Surveyor Moon Landers for gathering data about moon which was crucial before launching astronauts.

The following table summarises major milestones in US space exploration:

| Year | Mission/Event |
| ---  | ---           |
| 1961 | Announcement of the Apollo Program by Pre